In [ ]:
#######################################################################
# This script was written by Nana Owusu, it is meant to preprocess    #
# metabolomic information from CSV files and using the mean absolute  #
# deviation of each treatment group, remove outliers.                 #
#######################################################################
# Modules for text interpretation and math
import os, sys, re, fnmatch
import numpy as np
# Modules for plotting and reading csv
# import matplotlib.pyplot as plt
# from matplotlib.figure import Figure
# import csv
import pandas as pd
# %matplotlib inline
# from ipywidgets import interactive
# Module for GUI
# import tkinter as tk
# from tkinter import filedialog
# Module for saving plots as PDF
# from matplotlib.backends.backend_pdf import PdfPages

## Load csv files with pandas

In [ ]:
dataFile = pd.read_csv("/Users/nowusu/uiHackyHour/uihh_nowusu/20190723_chronicrotenoneisradipine_driftnorm.csv")

###     Group metabolite columns by drug treatment

In [ ]:
## variable that contains the names of each metabolite measured
# The syntax here is known as a "list comprehension"
columns = [col for col in dataFile.columns if col not in ["Condition", "sample"]]

# gives a Pandas Series of the 21 treatment drugs
groups = dataFile['Condition']

### Get metabolite values sorted into a multi-index format

In [ ]:
# Routine for numbering each individual sample
# treated with a particular drug. The final
# output is a list of tuples (a python data type defined by ())
drug_enumerate = [(groups[0],0)]
count = 0
oldDrug = groups[0]
for i in range(1,len(groups)):
    if oldDrug == groups[i]:
        count += 1
    else :
        count = 0
    drug_enumerate.append((groups[i],count))
    oldDrug = groups[i]


drug_multiIdx = pd.MultiIndex.from_tuples(drug_enumerate)

# the multi-index Pandas object will be added to this dataframe object
# for allow for better group analyses using pandas tools for stats.
metabolites_mi = pd.DataFrame(dataFile[columns])
metabolites_mi.set_axis(labels=drug_multiIdx, axis='index', inplace=True)

In [ ]:
metabolites_mi

## Functions for calculating group statistics

In [ ]:
def getConditions(condGroups):
    ''' Routine for counting how many constituents are in a sequence
    after the first occurrence and saves the constituent as well
    as the count '''
    
    drugs = {}
    for treatment in condGroups:
        if treatment not in drugs:
            drugs[treatment] = 0
        drugs[treatment] += 1
    
    return drugs

def stdErr(group,metabSet):
    # calculate standard deviation for
    # each group    
    metabStdErr = pd.concat([pd.DataFrame
                    (metabSet.loc[treatment,columns].std(axis='index')).T 
                             for treatment in group], ignore_index=True)
    
    metabStdErr.set_axis(axis='index', labels=group, inplace=True)
    metabStdErr.columns.names = ['Standard Deviation']
    return metabStdErr
    

def meanStdErr(group,metabSet):
    # calculate mean standard error 
    # of each group    
    metabMeanStdErr = pd.concat([pd.DataFrame
                    (metabSet.loc[treatment,columns].sem(axis='index')).T 
                             for treatment in group], ignore_index=True)
    
    metabMeanStdErr.set_axis(axis='index', labels=group, inplace=True)
    metabMeanStdErr.columns.names = ['Mean Std. Error']
    
    return metabMeanStdErr

def coefOfVar(group,metabStdErr,metabMean):
    # calculate coefficient of variation 
    # of each group    
    metabCoefOfVar = metabStdErr.truediv(other=metabMean,axis='index')
    
    metabCoefOfVar.columns.names = ['Coeff. of Variation']
    
    return metabCoefOfVar

def mean(group,metabSet):
    # calculate mean of each group    
    metabMean = pd.concat([pd.DataFrame
                    (metabSet.loc[treatment,columns].mean(axis='index')).T 
                             for treatment in group], ignore_index=True)
    
    metabMean.set_axis(axis='index', labels=group, inplace=True)
    metabMean.columns.names = ['Mean Std. Deviation']
    
    return metabMean

def grubbs(group,metabSet,metabMean,metabStdErr):
    # perform Grubb's analysis

    meanAbsDev = pd.DataFrame([])
    for treatment in group:
            operand = metabSet.loc[treatment,columns].sub \
                        (metabMean.loc[treatment,columns])
            operand = operand.abs()
            meanAbsDev = meanAbsDev.append(operand.div(metabStdErr.loc[treatment,columns]))
    
    meanAbsDev.set_axis(labels=drug_multiIdx,axis='index',inplace=True)
    meanAbsDev.columns.names = ['Mean Abs. Deviation']
    
    return meanAbsDev

## Calculate standard deviations

In [ ]:
conditions = getConditions(groups)
std = stdErr(conditions,metabolites_mi)
std

## Calculate Averages

In [ ]:
avg = mean(conditions,metabolites_mi)
avg

## Calculate Coefficient of Variation

In [ ]:
cv = coefOfVar(conditions,std,avg)
cv

## Perform Grubb's Analysis

In [ ]:
madVals = grubbs(conditions,metabolites_mi,avg,std)
madVals

## Function for determining outliers

In [ ]:
def outliers(grubbsData,initVals,thresh):
    
    # If the condition above is true, replace the value with NaN
    testDF = grubbsData.gt(thresh)
    valueCheck = grubbsData.mask(cond=testDF,other=np.nan)
    
    # if condition above is false, replace with previous value
    valueCheck = valueCheck.where(cond=testDF,other=initVals)
    
    return valueCheck

In [ ]:
whichVals = outliers(madVals,metabolites_mi,1.15)
whichVals

## Function for normalization

In [ ]:
def normalize(group,outly,meanVals):
    # Perform normalization
    nrmlz = pd.concat([pd.DataFrame
                    (outly.loc[treatment,columns].div(meanVals.loc[treatment]))
                     for treatment in group], ignore_index=True)
    
    nrmlz.set_axis(axis='index', labels=drug_multiIdx, inplace=True)
    nrmlz.columns.names = ['Normalization']
    
    return nrmlz

In [ ]:
meanOutly = mean(conditions,whichVals)
normal = normalize(conditions,whichVals,meanOutly)

## Function for exclusion

In [ ]:
def exclusion(grubbsData,normVals,thresh):
    
    # If the condition above is true, replace the value with NaN
    testDF = grubbsData.lt(thresh)
    exclude = grubbsData.mask(cond=testDF,other=normVals)
    
    # if condition above is false, replace with previous value
    exclude = exclude.where(cond=testDF,other=np.nan)
    
    return exclude

In [ ]:
remove = exclusion(madVals,normal,thresh)
remove